In [1]:
import os, sys
import torch
import signal
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from main import Main__

model_config = {'amp':True, 'use_distributed':False,'accum_grad_iters':1,
                'chat_template': True, 'end_sym': '\n', 'prompt_template': "<s>[INST] {} [/INST]",
                'max_txt_len': 1024, 'max_context_len': 3500,
                'ouput_dir': './exper01_llama2',
                'stage_ckpt': '/ssd3/chih/LLM/MiniGPT-4-ckpt/checkpoint_stage3.pth', 
                'vis_root_train': './dataset/minigpt_casing_train/coco/image/train',
                'ann_paths_train': ['./dataset/minigpt_casing_train/coco_caption/defe_ready_anno.json'],
                'vis_root_valid': './dataset/minigpt_casing_test/coco/image/test',
                'ann_paths_valid': ['./dataset/minigpt_casing_test/coco_caption/defe_ready_anno.json']}

llm_config = {'llama_model':'/ssd3/chih/LLM/Llama-2-7b-chat-hf', 'low_resource':False, 'low_res_device':0, 
              'lora_r':64, 'lora_target_modules':["q_proj", "v_proj"], 'lora_alpha':16,'lora_dropout':0.05
              }
# llm_config = {'llama_model':'/ssd3/chih/LLM/Meta-Llama-3-8B-Instruct', 'low_resource':False, 'low_res_device':0, 
#               'lora_r':64, 'lora_target_modules':["q_proj", "v_proj"], 'lora_alpha':16,'lora_dropout':0.05
#               }
# '/ssd3/chih/LLM/Meta-Llama-3-8B-Instruct'


vit_config = {'model_name':'eva_clip_g', 'image_size': 448,  'drop_path_rate': 0, 'use_grad_checkpoint': True, 'vit_precision': 'fp16', 'freeze_vit': True, }

lr_config = {'init_lr': 1e-5, 'beta2':0.999,'min_lr': 1e-6, 'decay_rate': None, 'weight_decay':0.05,
                'warmup_start_lr': 1e-6, 'warmup_steps': 1000, 'iters_per_epoch': 1000}


main_ = Main__(model_config=model_config)
main_.VLM_build(llm_config=llm_config, vit_config=vit_config)

Loading LLAMA
tokenizer pass


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 33554432 || all params: 6771970048 || trainable%: 0.49548996469513035
Position interpolate from 16x16 to 32x32


In [2]:
from utils.caption_datasets import COCOCaptionDataset
from torch.utils.data import DataLoader
# demo測試對話反饋
train_data_set = COCOCaptionDataset(vis_root=model_config['vis_root_valid'], 
                                ann_paths=model_config['ann_paths_valid'],
                                img_size=448)
dataloader =DataLoader(train_data_set, batch_size=1, num_workers=1, shuffle=True, pin_memory=True)
ckpt_url = '/ssd3/chih/VLM_Rebuild/exper01_llama2/checkpoint_best.pth' 
main_.load_checkpoint(ckpt_url=ckpt_url)

ann paths ['./dataset/minigpt_casing_test/coco_caption/defe_ready_anno.json']
{'caption': 'Defective.', 'image': '/ssd3/chih/Dataset/minigpt_casing_test/coco/image/test/casting512x512_deffront_0.jpeg', 'image_id': 'casting512x512_deffront_0.jpeg', 'instance_id': '0'}
./dataset/minigpt_casing_test/coco/image/test/casting512x512_okfront_1169.jpeg 



In [3]:
samples = next(iter(dataloader))
samples['image'] = samples['image'].cuda()
ans = main_.chat_module(samples=samples)
print("Q:",samples['instruction_input'], "answer:",samples['answer'])
print("A:",ans[0])

/ssd1/chihyi/miniconda3/envs/minigptv/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/ssd1/chihyi/miniconda3/envs/minigptv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Q: ['<Img><ImageHere></Img> [caption] Does this photo have any flaws? '] answer: ['defective']
A: 
